In [11]:
import docplex.cp
from docplex.cp.model import CpoModel, CpoStateFunction, CpoStepFunction

from reader import read
from Pizza import Pizza

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
fname = "e_elaborate.in.txt"

clients, ingredients = read(f'data/{fname}')

nb_clients, nb_ing = len(clients), len(ingredients)
CLI = range(nb_clients)
ING = range(nb_ing)

In [25]:
model = CpoModel("pizza")

X_i = []
for i in ING:
    X_i.append( model.binary_var(name=f'X[{i}]') )

C_j = []
for j in CLI:
    C_j.append( model.binary_var(name=f'C[{j}]') )

for j, c in enumerate(clients):
    
    bool_likes = [X_i[ing] == 1 for ing in c[0]]
    bool_dislikes = [ X_i[ing] == 0 for ing in c[1] ]
    
    bool_conds = bool_likes + bool_dislikes
    
    model.add( C_j[j] == model.logical_and(bool_conds) )
    
ing_like_clients = {}
ing_dislike_clients = {}
for j, c in enumerate(clients):
    for ing in c[0]:
        ing_like_clients.setdefault(ing, []).append(j)
    for ing in c[1]:
        ing_dislike_clients.setdefault(ing, []).append(j)

conflicted_clients = {}
for ing in ing_like_clients:
    for c in ing_like_clients[ing]:
        for other in ing_dislike_clients.get(ing, []):
            conflicted_clients.setdefault(c,set()).add(other)

for ing in ing_like_clients:
    model.add( model.if_then( sum(C_j[c] for c in ing_like_clients[ing]) >= 1,
                            sum(C_j[other] for other in ing_dislike_clients.get(ing,[])) == 0 ) )
    
"""for c in conflicted_clients:
    model.add( model.if_then( C_j[c] == 1, sum(C_j[other] for other in conflicted_clients[c]) == 0 ) )"""

model.add(model.maximize( sum(C_j[j] for j in CLI)) )

In [26]:
msol = model.solve( LogVerbosity="Normal", LogPeriod = 1000000,
                    trace_log=True,  add_log_to_solution= True)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Maximization problem - 14986 variables, 14986 constraints
 ! Presolve      : 2 extractables eliminated
 ! LogPeriod            = 1000000
 ! Initial process time : 0.51s (0.51s extraction + 0.00s propagation)
 !  . Log search space  : 14800.0 (before), 14800.0 (after)
 !  . Memory usage      : 14.1 MB (before), 14.1 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0      14800                 -
 + New bound is 4986
                        0      14800    1            -
 + New bound is 4985
                        2      14800    1   F     0 != X[2724]
 + New bound is 4984
 *          1262     7589  12.08s       3      (gap is 294.9%)
 *          1313     7461  12.08s      12      (gap is 279.6%)
 *          1314    18490  24.31s       8

KeyboardInterrupt: 